In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

2024-09-29 10:36:37.801700: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-29 10:36:38.036393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 10:36:38.121882: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 10:36:38.145328: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 10:36:38.362619: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Schedules
[name for name in dir(keras.optimizers.schedules) if not name.startswith("_")]

['CosineDecay',
 'CosineDecayRestarts',
 'ExponentialDecay',
 'InverseTimeDecay',
 'LearningRateSchedule',
 'PiecewiseConstantDecay',
 'PolynomialDecay',
 'deserialize',
 'serialize']

In [3]:
# 1. Polynomial Decay
lr_poly = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    end_learning_rate=0.0001,
    power=0.5,
    cycle=False,
)

In [4]:
# 2. Exponential Decay
Ir_exp = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1, decay_steps=100000, decay_rate=0.96
)

In [5]:
# 3. Piecewise Constant Decay
lr_ped = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[100000, 110000],  # s值
    values=[1.0, 0.5, 0.1],
)
# 當訓練次數<100000, values代入 1.0
# 介於100000, 110000, values代入 0.5
# >100000, values代入 0.11

In [6]:
# Ex 練習
# Load data
from tensorflow.keras.datasets import fashion_mnist

(x_train_set, y_train_set), (x_test, y_test) = fashion_mnist.load_data()

# Split data
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_set, y_train_set, random_state=1
)

# Preprocessing
# (45000,28,28)->(1,28,28) / keepdims=True 保持原維度，否則會變成二維(28,28)
pixel_means = x_train.mean(axis=0, keepdims=True)
pixel_stds = x_train.std(axis=0, keepdims=True)

x_train_scaled = (x_train - pixel_means) / pixel_stds
x_valid_scaled = (x_valid - pixel_means) / pixel_stds
x_test_scaled = (x_test - pixel_means) / pixel_stds

In [8]:
model = keras.models.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
        keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
        keras.layers.Dense(10, activation="softmax"),
    ]
)

/home/chesterxalan/github/Classwork/Classwork-PythonDLApplicationDevelopment/.venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1727577425.558434    5008 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727577425.726184    5008 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727577425.726235    5008 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727577425.730233    5008 c

In [9]:
lr_exp = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1, decay_steps=100000, decay_rate=0.96
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(learning_rate=lr_exp),
    metrics=["accuracy"],
)


In [10]:
train = model.fit(
    x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid)
)


Epoch 1/2


I0000 00:00:1727577432.813271    5213 service.cc:146] XLA service 0x7fce38019450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727577432.813321    5213 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-09-29 10:37:12.842788: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-29 10:37:12.906940: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


  88/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6050 - loss: 1.6787

I0000 00:00:1727577433.481412    5213 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7819 - loss: 0.6877 - val_accuracy: 0.7559 - val_loss: 0.7301
Epoch 2/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8579 - loss: 0.3954 - val_accuracy: 0.8005 - val_loss: 0.5858
